In [1]:
!pip install transformers==3.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.0/769.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 10.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 59.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [2]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
train = pd.read_csv('/content/FAANG_STOCK_NEWS.csv')

In [5]:
train.head()

,Unnamed: 0,ticker,date,time,headline,neg,neu,pos,compound,summary
0,0,FB,9/22/2021,06:30AM,2 Growth Stocks to Buy Hand Over Fist If the M...,0.000,0.608,0.392,0.7003,1
1,1,FB,9/22/2021,02:23AM,UPDATE 1-Facebook wraps up deals with Australi...,0.167,0.833,0.000,-0.3400,1
2,2,FB,9/22/2021,12:36AM,Facebook wraps up deals with Australian media ...,0.179,0.821,0.000,-0.3400,1
3,3,FB,9/22/2021,12:27AM,Facebook wraps up deals with Australia media f...,0.000,1.000,0.000,0.0000,1
4,4,FB,9/21/2021,10:34PM,Facebook overpaid FTC fine as quid pro quo to ...,0.099,0.659,0.242,0.3818,1


In [6]:
train['Sentiment'] = train.loc[:,['neg', 'neu','pos']].max(axis=1)



In [7]:
def generating_category(negative, neu, positive,sentiment):
  if sentiment == negative:
    return 0
  if sentiment == neu:
    return 1
  else :
    return 2
train_updated = train;
train_updated['Sentiment_class']=train.apply(lambda x: generating_category(x['neg'],x['neu'],x['pos'],x['Sentiment']) , axis=1)

In [8]:

new_df = train_updated[['headline', 'Sentiment_class']]

In [9]:
print(train_updated.info())
print(train_updated.describe())
new_df.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       500 non-null    int64  
 1   ticker           500 non-null    object 
 2   date             500 non-null    object 
 3   time             500 non-null    object 
 4   headline         500 non-null    object 
 5   neg              500 non-null    float64
 6   neu              500 non-null    float64
 7   pos              500 non-null    float64
 8   compound         500 non-null    float64
 9   summary          500 non-null    int64  
 10  Sentiment        500 non-null    float64
 11  Sentiment_class  500 non-null    int64  
dtypes: float64(5), int64(3), object(4)
memory usage: 47.0+ KB
None
       Unnamed: 0         neg         neu         pos    compound     summary  \
count  500.000000  500.000000  500.000000  500.000000  500.000000  500.000000   
mean   249.500000    

headline                                                                                        Sentiment_class
Emmy Award ratings surprise to the upside, boosted by big wins from 'The Crown,' 'Ted Lasso'    1                  6
Uncertainty around taxes could cause stocks to pullback this month, strategist warns            1                  5
Why Goldman Sachs just launched this ETF that will rival FAANG stocks                           1                  5
Google, Apple remove Navalny app from stores as Russian elections begin                         1                  4
FTC, Biden Antitrust Enforcement Push Takes On Amazon, Google  And The Supreme Court            1                  4
                                                                                                                  ..
Exclusive-Amazon faces Teamsters union drive at nine Canadian sites                             1                  1
Evaluating Apple (AAPL) After Epic Antitrust Decision                

In [10]:
new_df.head()

,headline,Sentiment_class
0,2 Growth Stocks to Buy Hand Over Fist If the M...,1
1,UPDATE 1-Facebook wraps up deals with Australi...,1
2,Facebook wraps up deals with Australian media ...,1
3,Facebook wraps up deals with Australia media f...,1
4,Facebook overpaid FTC fine as quid pro quo to ...,1


In [11]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [12]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.headline
        self.targets = self.data.Sentiment_class
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [13]:
train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
testing_set = SentimentData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (500, 2)
TRAIN Dataset: (400, 2)
TEST Dataset: (100, 2)


In [14]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [15]:
torch.nn.ReLU()

ReLU()

In [16]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.l2_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.pre_classifier = torch.nn.Linear(768, 3)
        self.classifier = torch.nn.ReLU()


    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.l2_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        pooler = self.pre_classifier(pooler)
        output = self.classifier(pooler)
        return output

In [17]:
model = RobertaClass()
model.to(device)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [18]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [19]:
## define functinn to check accuracy
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    print((preds==targets).sum())
    return n_correct

In [20]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for round,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if round%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

In [21]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tensor(1)
Training Loss per 5000 steps: 1.1311982870101929
Training Accuracy per 5000 steps: 12.5


1it [00:15, 15.85s/it]

tensor(0)


2it [00:31, 15.62s/it]

...

tensor(7)


49it [11:42, 14.20s/it]

tensor(8)


50it [11:56, 14.32s/it]

The Total Accuracy for Epoch 0: 87.75
Training Loss Epoch: 0.5105938121676445
Training Accuracy Epoch: 87.75


In [22]:
def validation(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    with torch.no_grad():
        for round, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            tm = calcuate_accuracy(big_idx, targets)
            n_correct += tm
            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            if round%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 5000 steps: {loss_step}")
                print(f"Validation Accuracy per 5000 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    return epoch_accu

In [23]:
accuracy = validation(model, testing_loader)
print("Accuracy on test data = %0.3f%%" % accuracy)

1it [00:02,  2.22s/it]

tensor(4)
Validation Loss per 5000 steps: 0.049012620002031326
Validation Accuracy per 5000 steps: 100.0


2it [00:04,  2.43s/it]

tensor(4)


3it [00:07,  2.41s/it]

...

tensor(4)


24it [00:55,  2.37s/it]

tensor(4)


25it [00:57,  2.31s/it]

tensor(4)
Validation Loss Epoch: 0.0856359525024891
Validation Accuracy Epoch: 99.0
Accuracy on test data = 99.000%


In [24]:
val_data = test_data[['headline']]

In [25]:
val_data = val_data.iloc[:4,:]

In [26]:
val_data

,headline
0,2 Growth Stocks to Buy Hand Over Fist If the M...
1,UPDATE 1-Facebook wraps up deals with Australi...
2,Facebook wraps up deals with Australia media f...
3,Facebook Oversight Board to Review Special Rul...


In [171]:
class SentimentDataPrediction(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.headline
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

In [165]:
##Ignoring warning
import warnings
warnings.filterwarnings("ignore")

In [172]:
data_to_predict_sentiment = SentimentDataPrediction(val_data, tokenizer, MAX_LEN)

In [173]:

def make_prediction(model,input_text_data):
  model.eval()
  with torch.no_grad():
    for index in range(input_text_data.data.shape[0]):
      text_data = input_text_data.text[index]
      data = input_text_data[index]
      ids = data['ids'].to(device, dtype = torch.long)
      mask = data['mask'].to(device, dtype = torch.long)
      token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
      ids = torch.tensor(ids,dtype=torch.long).reshape(1,256)
      mask = torch.tensor(mask,dtype=torch.long).reshape(1,256)
      # token_type_ids = torch.tensor(token_type_ids,dtype=torch.long).reshape(1,256)
      # ids=data['ids'].clone().detach()
      # mask=mask.clone().detach().requires_grad_(False)
      token_type_ids = data['token_type_ids'].clone().detach().requires_grad_(False)
      outputs = model(ids, mask, token_type_ids)
      predicted_value = torch.max(outputs.data, dim=1).indices
      print(text_data)
      if (predicted_value.item() == 0 ) :
        print("Negative")
      elif (predicted_value.item() == 1) :
        print("Neutral")
      else :
        print("Positive")

make_prediction(model,data_to_predict_sentiment)

2 Growth Stocks to Buy Hand Over Fist If the Market Crashes
Neutral
UPDATE 1-Facebook wraps up deals with Australian media firms, TV broadcaster SBS excluded
Neutral
Facebook wraps up deals with Australia media firms, TV broadcaster SBS not included
Neutral
Facebook Oversight Board to Review Special Rules for High-Profile Users
Neutral


In [151]:

predict_params = {'batch_size': 1,
                'shuffle': True,
                'num_workers': 0
                }

def prediction(model,info_loader):
  model.eval()
  with torch.no_grad():
        for round, data in tqdm(enumerate(info_loader, 0)):
          ids = data['ids'].to(device, dtype = torch.long)
          mask = data['mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
          print("ids" ,ids.shape)
          print("mask" , token_type_ids.shape)

          print(info_loader)
          outputs = model(ids, mask, token_type_ids)
          predicted_value = torch.max(outputs.data, dim=1).indices
          print(predicted_value)
          break


readytopredict = DataLoader(xyz, **predict_params)
prediction(model,readytopredict)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


ids torch.Size([1, 256])
mask torch.Size([1, 256])


0it [00:00, ?it/s]

tensor([1])


In [ ]:
tokenizer.decode()

In [126]:
output_model_file = 'pytorch_roberta_sentiment.bin'
output_vocab_file = './'

model_to_save = model
torch.save(model_to_save, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

('./vocab.json', './merges.txt')

In [ ]:
## Labeling sentimentvalues

def map_sentiment_class(val):
  if val == 0:
    return "neutral"
  elif val == 1:
    return "negative"
  else:
    return "Positive"



In [153]:
input_text = "2 Growth Stocks to Buy Hand Over Fist If the M"
input_text_tokenized = tokenizer.encode_plus(
            input_text,
            None,
            add_special_tokens=True,
            max_length=MAX_LEN,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
# print(input_text_tokenized)

print(tokenizer.decode([   0,  7608,  1645,    16, 12812,     5,   382,   168,     7, 25150,
        16062,     2,     1,     1,     1]))

<s>Why Amazon is lobbying the US government to legalize weed</s><pad><pad><pad>
